In [1]:
%matplotlib inline
import os
path = '/root/github/3_SpecialModule'
os.sys.path.append(path)

from Me import *

Sucessful Import Me!!!


# Train Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Params

In [3]:
learning_rate = 0.01
training_epochs = 20
batch_size = 256
display_step = 1
examples_to_show = 10
total_batch = int(mnist.train.num_examples/batch_size)

# Construct Model

In [4]:
# Input
tf_x = tf.placeholder(tf.float32, [None, 784])
tf_y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# neural network layers
W1 = tf.Variable(tf.truncated_normal([784, 300], stddev=0.1))
b1 = tf.Variable(tf.zeros([300]))
W2 = tf.Variable(tf.zeros([300, 10]))
b2 = tf.Variable(tf.zeros([10]))

hidden1 = tf.nn.relu(tf.matmul(tf_x, W1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2)

# Loss + Optimizer

In [5]:
loss = tf.reduce_mean(-tf.reduce_sum(tf_y* tf.log(y), reduction_indices=[1]))
optimizer = tf.train.AdagradOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

accuracy = tf.metrics.accuracy(labels=tf.argmax(tf_y, axis=1), predictions=tf.argmax(y, axis=1),)[1] # accuracy

# Initializing the variables

In [6]:
sess = tf.Session()                                                                 # control training and others
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)     # initialize var in graph

# Launch the graph

In [8]:
training_epochs = 5
batch_size = 256
display_step = 1
traindata = mnist.train
testdata = mnist.test
total_batch = int(mnist.train.num_examples/batch_size)

# Train model
for step in range(training_epochs):
    for i in range(total_batch):
        batch_x, batch_y = traindata.next_batch(batch_size)
        _, l, acc, pred = sess.run([train_step, loss, accuracy, y],{tf_x: batch_x, tf_y: batch_y, keep_prob: 0.75})
    if step % display_step == 0:
        print("===> Epoch: {:04d}, loss = {:0.9f}, accuracy = {:0.9f}".format(step + 1, l, acc))
print ("Optimization Finished!")

# Test model
# correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(tf_y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# accuracy = tf.metrics.accuracy(labels=tf.argmax(tf_y, axis=1), predictions=tf.argmax(y, axis=1),)[1] # 与上等价，前面已做

print("Accuracy: {:0.9f}".format(sess.run(accuracy, {tf_x: testdata.images, tf_y: testdata.labels, keep_prob: 1.0})))

===> Epoch: 0001, loss = 0.581252098, accuracy = 0.803240120
===> Epoch: 0002, loss = 0.392780900, accuracy = 0.822244227
===> Epoch: 0003, loss = 0.370405048, accuracy = 0.835155547
===> Epoch: 0004, loss = 0.372393757, accuracy = 0.844515145
===> Epoch: 0005, loss = 0.301665753, accuracy = 0.852029145
Optimization Finished!
Accuracy: 0.853868246


In [9]:
# 多一层循环
# 一轮相当于：total_batch*batch_size
# 一层循环的时候，training_epochs*batch_size/mnist.train.num_examples相当于迭代（epoch）次数

